In [15]:
path=str(input("Enter the path : "))
a="\\"
b="/"
newpath=path.replace(a,b)
print(newpath)    

C:/Users/Moksha's Lappy/Desktop/GFG/electronics-10-01348-v2.pdf


In [16]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

text1=extract_text_from_pdf(newpath)

In [2]:
import spacy
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy model for entity extraction
nlp = spacy.load("en_core_web_sm")

# News API configuration
API_KEY = "62f819f2173c4068b3d9a4644b487d0c"  # Replace with your News API key

def get_user_article():
    """
    Prompts the user to input the title and content of the article.
    """
    title = input("Enter the article title: ")
    content = input("Enter the article content: ")
    return title, content

def extract_entities(text):
    """
    Extracts named entities like PERSON, ORG, and GPE (Geo-political entities) from the article.
    """
    doc = nlp(text)
    entities = {
        "PERSON": [],
        "ORG": [],
        "GPE": []
    }
    for ent in doc.ents:
        if ent.label_ in entities:
            entities[ent.label_].append(ent.text)
    return entities

def compute_similarity(text1, text2):
    """
    Computes the cosine similarity between two articles using TF-IDF.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity_score

def compute_confidence_score(similarity, source_reliability):
    """
    Computes a confidence score based on similarity and source reliability.
    """
    return (0.7 * similarity) + (0.3 * source_reliability)

def fetch_similar_articles(query):
    """
    Fetches news articles related to a given query from News API.
    """
    search_url = f"https://newsapi.org/v2/everything?q={query}&apiKey={API_KEY}"
    response = requests.get(search_url)
    if response.status_code == 200:
        return response.json()["articles"][:5]  # Return top 5
    return []

# Main program
title, content = get_user_article()

# Step 2: Extract entities from the user's article
entities = extract_entities(content)
print("Extracted Entities:", entities)

# Step 3: Compare similarity with trusted sources (fetch similar articles)
similar_articles = fetch_similar_articles(title)

# Print top 5 similar articles
print("\nTop 5 Similar Articles:")
for article in similar_articles:
    print(f"Title: {article['title']}")
    print(f"URL: {article['url']}")
    print("-" * 50)

# Compute the similarity with one of the similar articles (e.g., the first one)
if similar_articles:
    similar_article_content = similar_articles[0]["content"]  # Fetch content of the first similar article
    similarity_score = compute_similarity(content, similar_article_content)
    print(f"Similarity with first similar article: {similarity_score:.2f}")

    # Step 4: Compute the confidence score
    source_reliability = 0.9  # Assuming high reliability for the trusted source
    confidence_score = compute_confidence_score(similarity_score, source_reliability)
    print(f"Confidence Score: {confidence_score * 100:.2f}%")


Extracted Entities: {'PERSON': [], 'ORG': [], 'GPE': []}

Top 5 Similar Articles:
Title: A new outlet covers climate policy in the language Brazil knows best: Soccer.
URL: https://www.niemanlab.org/2025/02/a-new-outlet-covers-climate-policy-in-the-language-brazil-knows-best-soccer/
--------------------------------------------------
Title: Ranked! The Top 100 women's footballers of all-time
URL: https://www.fourfourtwo.com/news/ranked-top-100-womens-footballers-of-all-time
--------------------------------------------------
Title: Everything you need to know about sports tourism
URL: https://www.nationalgeographic.com/travel/article/why-sports-tourism-is-on-the-rise
--------------------------------------------------
Title: A 2025 Sports Calendar for Journalists
URL: https://mediablog.prnewswire.com/2025/01/09/2025-sports-calendar/
--------------------------------------------------
Similarity with first similar article: 0.06
Confidence Score: 31.12%


: 